In [1]:
import pulp
import pandas as pd

In [2]:
def montar_instancia():
    turmas = ["1A", "2A", "3A"]
    ano_turma = {"1A": 1, "2A": 2, "3A": 3}

    disciplinas = ["Matematica", "Fisica", "Quimica", "Inovativa"]

    professores = ["prof1", "prof2", "prof3"]

    horarios = ["Seg_M", "Ter_M", "Qua_M", "Qui_M", "Sex_M",
                "Seg_T", "Ter_T", "Qua_T", "Qui_T", "Sex_T"]

    salas = ["Sala_1", "Sala_2", "Sala_3"]

    professor_pode_lecionar = {
        ("prof1", "Matematica"): 1,
        ("prof2", "Fisica"): 1,
        ("prof3", "Quimica"): 1,
        ("prof1", "Inovativa"): 1,
    }

    disponibilidade = {(p, h): 1 for p in professores for h in horarios}

    quantidade_aulas = {}
    for turma in turmas:
        quantidade_aulas[(turma, "Matematica")] = 1
        quantidade_aulas[(turma, "Fisica")] = 1
        quantidade_aulas[(turma, "Quimica")] = 1
        quantidade_aulas[(turma, "Inovativa")] = 1

    disciplinas_programas = {"Inovativa"}
    disciplinas_programas_externos = {}

    atribuicoes_fixas = {}

    return {
        "turmas": turmas,
        "ano_da_turma": ano_turma,
        "disciplinas": disciplinas,
        "professores": professores,
        "horarios": horarios,
        "salas": salas,
        "professor_pode_lecionar": professor_pode_lecionar,
        "professor_disponivel_no_horario": disponibilidade,
        "quantidade_aulas": quantidade_aulas,
        "atribuicoes_fixas": atribuicoes_fixas,
        "disciplinas_programas": disciplinas_programas,
        "disciplinas_programas_externos": disciplinas_programas_externos,
    }


In [3]:
def construir_e_resolver_modelo(instancia, limite_tempo=120):
    turmas = instancia["turmas"]
    ano_da_turma = instancia["ano_da_turma"]
    disciplinas = instancia["disciplinas"]
    professores = instancia["professores"]
    horarios = instancia["horarios"]
    salas = instancia["salas"]

    professor_pode_lecionar = instancia["professor_pode_lecionar"]
    professor_disponivel_no_horario = instancia["professor_disponivel_no_horario"]

    quantidade_aulas = {
        (t, d): instancia["quantidade_aulas"].get((t, d), 0)
        for t in turmas
        for d in disciplinas
    }

    atribuicoes_fixas = set(instancia.get("atribuicoes_fixas", []))
    disciplinas_programas = set(instancia.get("disciplinas_programas", []))
    disciplinas_programas_externos = set(instancia.get("disciplinas_programas_externos", []))

    #  VARIÁVEIS DE DECISÃO
    #  x[turma, disciplina, professor, horario, sala] = 1 se a aula ocorre

    # Objetivo zero (somente factibilidade)
    problema = pulp.LpProblem("Alocacao_Aulas_Escolar")
    problema += 0, "Objetivo_Nulo"

    variavel_alocacao = {}

    for turma in turmas:
        for disciplina in disciplinas:
            for professor in professores:
                for horario in horarios:
                    for sala in salas:

                        if (turma, disciplina, professor, horario, sala) in atribuicoes_fixas:
                            continue

                        if professor_pode_lecionar.get((professor, disciplina), 0) == 0:
                            continue

                        if professor_disponivel_no_horario.get((professor, horario), 0) == 0:
                            continue

                        variavel_alocacao[(turma, disciplina, professor, horario, sala)] = (
                            pulp.LpVariable(
                                f"alocar_{turma}_{disciplina}_{professor}_{horario}_{sala}",
                                cat="Binary"
                            )
                    )

    #  RESTRIÇÃO 1 — COBERTURA DAS AULAS
    for turma in turmas:
        for disciplina in disciplinas:

            problema += (
                pulp.lpSum(
                    variavel_alocacao.get((turma, disciplina, professor, horario, sala), 0)
                    for professor in professores
                    for horario in horarios
                    for sala in salas
                ) == quantidade_aulas[(turma, disciplina)],
                f"cobertura_{turma}_{disciplina}"
            )

    #  RESTRIÇÃO 2 — UMA AULA POR TURMA POR HORÁRIO
    for turma in turmas:
        for horario in horarios:

            problema += (
                pulp.lpSum(
                    variavel_alocacao.get((turma, disciplina, professor, horario, sala), 0)
                    for disciplina in disciplinas
                    for professor in professores
                    for sala in salas
                ) <= 1,
                f"uma_aula_por_turma_{turma}_{horario}"
            )

    #  RESTRIÇÃO 3 — UM PROFESSOR POR HORÁRIO
    for professor in professores:
        for horario in horarios:

            problema += (
                pulp.lpSum(
                    variavel_alocacao.get((turma, disciplina, professor, horario, sala), 0)
                    for turma in turmas
                    for disciplina in disciplinas
                    for sala in salas
                ) <= 1,
                f"uma_aula_por_professor_{professor}_{horario}"
            )

    #  RESTRIÇÃO 4 — LIMITE DE SALAS POR HORÁRIO
    for sala in salas:
        for horario in horarios:

            problema += (
                pulp.lpSum(
                    variavel_alocacao.get((turma, disciplina, professor, horario, sala), 0)
                    for turma in turmas
                    for disciplina in disciplinas
                    for professor in professores
                ) <= 1,
                f"uma_aula_por_sala_{sala}_{horario}"
            )
    
    #  REGRAS POR ANO (PROGRAMAS INTERNOS E EXTERNOS)
    for turma in turmas:
        ano = ano_da_turma[turma]

        variaveis_programas = pulp.lpSum(
            variavel_alocacao.get((turma, disc, professor, horario, sala), 0)
            for disc in disciplinas_programas
            for professor in professores
            for horario in horarios
            for sala in salas
        )

        if ano == 1:
            problema += (
                variaveis_programas >= 1,
                f"ano1_precisa_programa_{turma}"
            )

        if ano == 2:
            variaveis_programas_externos = pulp.lpSum(
                variavel_alocacao.get((turma, disc, professor, horario, sala), 0)
                for disc in disciplinas_programas_externos
                for professor in professores
                for horario in horarios
                for sala in salas
            )

            problema += (
                variaveis_programas_externos <= 1,
                f"ano2_limite_programa_externo_{turma}"
            )

        if ano == 3:
            for disc_ext in disciplinas_programas_externos:
                for professor in professores:
                    for horario in horarios:
                        for sala in salas:
                            if (turma, disc_ext, professor, horario, sala) in variavel_alocacao:
                                problema += (
                                    variavel_alocacao[(turma, disc_ext, professor, horario, sala)] == 0,
                                    f"ano3_proibido_programa_externo_{turma}_{disc_ext}_{horario}_{sala}"
                                )

    #  SOLVER
    solver = pulp.PULP_CBC_CMD(msg=True, timeLimit=limite_tempo, threads=2)
    status = problema.solve(solver)

    print("\nSTATUS DO SOLVER:", pulp.LpStatus[problema.status])

    atribuicoes = []

    # Atribuições fixas
    for (turma, disc_ext, professor, horario) in atribuicoes_fixas:
        atribuicoes.append({
            "turma": turma,
            "disciplina": disc_ext,
            "professor": professor,
            "horario": horario,
            "sala": sala,
            "fixa": True
        })

    # Atribuições decididas pelo modelo
    for chave, var in variavel_alocacao.items():
        if pulp.value(var) is not None and pulp.value(var) > 0.5:
            turma, disc_ext, professor, horario, sala = chave
            atribuicoes.append({
                "turma": turma,
                "disciplina": disc_ext,
                "professor": professor,
                "horario": horario,
                "sala": sala,
                "fixa": False
            })

    df = pd.DataFrame(atribuicoes)
    if not df.empty:
        df = df.sort_values(["horario", "turma", "disciplina"]).reset_index(drop=True)

    return problema, df

In [4]:
instancia = montar_instancia()
instancia

{'turmas': ['1A', '2A', '3A'],
 'ano_da_turma': {'1A': 1, '2A': 2, '3A': 3},
 'disciplinas': ['Matematica', 'Fisica', 'Quimica', 'Inovativa'],
 'professores': ['prof1', 'prof2', 'prof3'],
 'horarios': ['Seg_M',
  'Ter_M',
  'Qua_M',
  'Qui_M',
  'Sex_M',
  'Seg_T',
  'Ter_T',
  'Qua_T',
  'Qui_T',
  'Sex_T'],
 'salas': ['Sala_1', 'Sala_2', 'Sala_3'],
 'professor_pode_lecionar': {('prof1', 'Matematica'): 1,
  ('prof2', 'Fisica'): 1,
  ('prof3', 'Quimica'): 1,
  ('prof1', 'Inovativa'): 1},
 'professor_disponivel_no_horario': {('prof1', 'Seg_M'): 1,
  ('prof1', 'Ter_M'): 1,
  ('prof1', 'Qua_M'): 1,
  ('prof1', 'Qui_M'): 1,
  ('prof1', 'Sex_M'): 1,
  ('prof1', 'Seg_T'): 1,
  ('prof1', 'Ter_T'): 1,
  ('prof1', 'Qua_T'): 1,
  ('prof1', 'Qui_T'): 1,
  ('prof1', 'Sex_T'): 1,
  ('prof2', 'Seg_M'): 1,
  ('prof2', 'Ter_M'): 1,
  ('prof2', 'Qua_M'): 1,
  ('prof2', 'Qui_M'): 1,
  ('prof2', 'Sex_M'): 1,
  ('prof2', 'Seg_T'): 1,
  ('prof2', 'Ter_T'): 1,
  ('prof2', 'Qua_T'): 1,
  ('prof2', 'Qui_T'): 

In [5]:
solucao, tabela = construir_e_resolver_modelo(instancia)

print("Objetivos:", pulp.value(solucao.objective))
print("Restrições:", len(solucao.constraints))
print("Variáveis:", len(solucao.variables()))

if tabela.empty:
    print("Nenhuma atribuição encontrada — instância possivelmente inviável ou precisa de tempo maior.")
else:
    print("\nGrade encontrada:")
    print(tabela.to_string(index=False))



STATUS DO SOLVER: Optimal
Objetivos: None
Restrições: 104
Variáveis: 361

Grade encontrada:
turma disciplina professor horario   sala  fixa
   2A  Inovativa     prof1   Qua_M Sala_3 False
   3A    Quimica     prof3   Qua_M Sala_1 False
   1A    Quimica     prof3   Qua_T Sala_2 False
   2A Matematica     prof1   Qua_T Sala_1 False
   2A     Fisica     prof2   Qui_M Sala_2 False
   3A Matematica     prof1   Qui_M Sala_1 False
   3A     Fisica     prof2   Qui_T Sala_2 False
   3A  Inovativa     prof1   Seg_M Sala_1 False
   2A    Quimica     prof3   Seg_T Sala_3 False
   1A     Fisica     prof2   Sex_M Sala_3 False
   1A  Inovativa     prof1   Ter_M Sala_2 False
   1A Matematica     prof1   Ter_T Sala_1 False
